In [1]:
! pip install transformers

In [2]:


import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline

In [3]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [4]:
#classifier = pipeline("zero-shot-classification", device=0)

### Get the Text Data

In [5]:
headlines = pd.read_csv('india-news-headlines.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'india-news-headlines.csv'

In [ ]:
df_head = headlines.head(100)

In [ ]:
samples = headlines[1:100]

In [ ]:
strings = samples.headline_text.values

In [ ]:
sequence = list(strings)

In [ ]:
print(sequence)

['Fissures in Hurriyat over Pak visit', "America's unwanted heading for India?", 'For bigwigs; it is destination Goa', 'Extra buses to clear tourist traffic', 'Dilute the power of transfers; says Riberio', 'Focus shifts to teaching of Hindi', 'IT will become compulsory in schools', "Move to stop freedom fighters' pension flayed", 'Gilani claims he applied for passport 2 years ago', "Parivar dismisses PM's warning", 'India; Pak exchange lists of N-plants', "Will Qureshi's return really help the govt?", "PM's tacit message: Put Ram tample on hold", "Text of the Prime Minister's article", 'NCW to focus on violence against women', "BBC's reality TV will focus on AIIMS", 'Jaitley firm on legal reforms', 'Hoshangabad farmers have enough water', 'BJP jumps into rail track conversion issue', "America's unwanted heading for India?", 'State may take AP to court over Krishna', 'This official has sued the briber', 'Park turns temple compound as residents watch', 'Moves afoot to evict squatters in 

# Classification

In [ ]:
candidate_labels = ["politics", "finance", "sports",'entertainment','agriculture','technology']


In [ ]:
df_head.head()

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic


In [ ]:
output = classifier(sequence, candidate_labels,multi_label = True)

In [ ]:
op = pd.DataFrame(output)

In [ ]:
op

,sequence,labels,scores
0,Fissures in Hurriyat over Pak visit,"[politics, finance, technology, entertainment,...","[0.6580215096473694, 0.010735863819718361, 0.0..."
1,America's unwanted heading for India?,"[politics, technology, finance, entertainment,...","[0.19470682740211487, 0.06738674640655518, 0.0..."
2,For bigwigs; it is destination Goa,"[entertainment, politics, finance, technology,...","[0.17034222185611725, 0.007796435616910458, 0...."
3,Extra buses to clear tourist traffic,"[technology, entertainment, sports, finance, p...","[0.18427591025829315, 0.05440380796790123, 0.0..."
4,Dilute the power of transfers; says Riberio,"[politics, finance, technology, agriculture, s...","[0.5691625475883484, 0.3738689422607422, 0.360..."
...,...,...,...
94,No report yet on fire in Udyog Bhavan,"[politics, finance, entertainment, technology,...","[0.026448814198374748, 0.02415291965007782, 0...."
95,He is the king of 'five star' industry,"[finance, technology, agriculture, entertainme...","[0.6690065264701843, 0.5558158755302429, 0.132..."
96,Petrol pump sealed for selling adulterated fuel,"[technology, finance, politics, entertainment,...","[0.12637867033481598, 0.013552123680710793, 0...."
97,Milk price up by Rs 1 per litre,"[finance, agriculture, entertainment, technolo...","[0.729042112827301, 0.27204886078834534, 0.064..."


### Converting the result into DataFrame with the maximum score

In [ ]:
output = []

filter_key1 = ['labels']
filter_key2 = ['scores']

for index, row in df_head.iterrows():
  d = {}
  seq = row['headline_text']
  result = classifier(seq, candidate_labels, multi_label=True)
 # print(result)
  temp_label = list(map(result.get, filter_key1))
  temp_score = list(map(result.get, filter_key2))
 # print(temp)
  d['headline_text'] = row['headline_text']
  d['label'] = temp_label[0][0]
  d['score'] = temp_score[0][0]
  output.append(d)


In [ ]:
 #convert the list of dictionary into pandas DataFrame
 new = pd.DataFrame(output)
 new

,headline_text,label,score
0,Status quo will not be disturbed at Ayodhya; s...,politics,0.239412
1,Fissures in Hurriyat over Pak visit,politics,0.658022
2,America's unwanted heading for India?,politics,0.194707
3,For bigwigs; it is destination Goa,entertainment,0.170342
4,Extra buses to clear tourist traffic,technology,0.184274
...,...,...,...
95,No report yet on fire in Udyog Bhavan,politics,0.026449
96,He is the king of 'five star' industry,finance,0.669008
97,Petrol pump sealed for selling adulterated fuel,technology,0.126384
98,Milk price up by Rs 1 per litre,finance,0.729043
